# Lab 11: SQL, FEC Data, and Small Donors

## Due: 11:59pm Wednesday 14th


In this lab, we explore the money donated during the 2016 election using the [Federal Election
Commission's public records](http://www.fec.gov/finance/disclosure/ftpdet.shtml). 

In [ ]:
import pandas as pd
import numpy as np
import re
import sqlalchemy

## Getting Started

For this assignment, you will be connecting to a sqlite database containing the data. The data we will be working with in this lab is quite small (16MB); however, it's taken from a larger database that more than few GBs!

### Connecting to the SQLite

In [ ]:
DB_LOCATION = '/tmp/fec_nyc.sqlite'

The following cell with get the database and verifies it exists.

In [ ]:
!wget https://s3.amazonaws.com/berkeley-ds100/fec_nyc.sqlite
!mv fec_nyc.sqlite $DB_LOCATION
!ls -lh $DB_LOCATION

The following cell will connect to the above database using the SQL Alchemy library.

In [ ]:
engine = sqlalchemy.create_engine(f"sqlite:///{DB_LOCATION}")
connection = engine.connect()

To check that everything is working we will list the available tables.

In [ ]:
engine.table_names()

## Table Descriptions

Here is a list of the tables in the database.  Each table links to the documentation on the [FEC page](http://www.fec.gov/finance/disclosure/ftpdet.shtml) for the dataset.

Note that the table names here are slightly different from the ones in lecture. Consult the FEC page
for the descriptions of the tables to find out what the correspondence is.

- [`cand`](http://www.fec.gov/finance/disclosure/metadata/DataDictionaryCandidateMaster.shtml): Candidates table. Contains names and party affiliation.
- [`comm`](http://www.fec.gov/finance/disclosure/metadata/DataDictionaryCommitteeMaster.shtml): Committees table. Contains committee names and types.
- [`indiv_sample_nyc`](http://www.fec.gov/finance/disclosure/metadata/DataDictionaryContributionsbyIndividuals.shtml): All individual contributions from New York City.


In [ ]:
# We use `LIMIT 5` to avoid displaying a huge table.
# Although our tables shouldn't get too large to display,
# this is generally good practice when working in the
# notebook environment.  Jupyter notebooks don't handle
# very large outputs well. 
query = """
SELECT * from indiv_sample_nyc LIMIT 5
"""
pd.read_sql(query, engine)

In [ ]:
query = '''
SELECT cand_id, cand_name
FROM cand
WHERE cand_pty_affiliation = 'DEM'
LIMIT 5
'''
pd.read_sql(query,engine)

### Notes on Grading 

1. For this lab, we will be grading your SQL expressions so be sure to do all computation in SQL (unless otherwise requested).  
1. Use the query string variable we provide since we will be analyzing your SQL query strings.

### Question 1: Donation Amount

### Question 1a

We are interested in finding the individuals from New York City that donated large sums. To begin to answer this question, we will look at the `indiv_sample_nyc` table. We'll find all the transactions that exceed \$5,000,000. However, if there are a lot of transactions like that, it might not be useful to list them all.  So before actually finding the transactions, _find out how many such transactions there are_. Use only SQL to compute the answer.

(It should be a table with a single column called **`count`** and a single entry, the number of transactions.)  

We will be grading the query string `query_q1a`.  You may modify our template but the result should contain the same information with the same names.

**In this question, and in the following ones, we will only be grading the output of** `pd.read_sql(query_qXX, engine)`.  
**Make sure that the output of this query matches what you want (order, column names, etc...)**

In [ ]:
# complete the query string
query_q1a = """
SELECT ...
FROM ...
WHERE ...
"""

### BEGIN SOLUTION
query_q1a = '''
SELECT count(*) as count
FROM indiv_sample_nyc
WHERE transaction_amt > 5000000
'''

### END SOLUTION
pd.read_sql(query_q1a, engine)

In [ ]:
res = pd.read_sql(query_q1a,engine)
assert len(res) == 1
assert res.columns == ['count']

assert res["count"][0] == 6

### Question 1b
Having seen that there aren't too many transactions that exceed 100,000 dollars, let's find them all.  Using only SQL, construct a table containing the committee ID, contributor's name, and the transaction amount, for transactions that exceed \$ 100,000 dollars.  Sort the transactions in decreasing order by amount. If two contributors contain the same transaction amount, sort by alphabetical order of their names.

We will be grading the query string `query_q1b`.  You may modify our template but the result should contain the same information with the same names.

The output should similar to the following:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>committee_id</th>
      <th>name</th>
      <th>transaction_amt</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>C00010603</td>
      <td>HILLARY VICTORY FUND</td>
      <td>23787000</td>
    </tr>
    <tr>
      <th>1</th>
      <td>C00571372</td>
      <td>C.V. STARR &amp; CO INC</td>
      <td>10000000</td>
    </tr>
    <tr>
      <th>2</th>
      <td>C00580100</td>
      <td>TRUMP, DONALD J.</td>
      <td>10000000</td>
    </tr>
    <tr>
      <th>3</th>
      <td>C00532705</td>
      <td>BLOOMBERG, MICHAEL R.</td>
      <td>8500000</td>
    </tr>
    <tr>
      <th>4</th>
      <td>C00010603</td>
      <td>HILLARY VICTORY FUND</td>
      <td>6175000</td>
    </tr>
    <tr>
      <th>5</th>
      <td>C00495861</td>
      <td>SOROS, GEORGE</td>
      <td>6000000</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q1b = """
SELECT 
    ... AS committee_id,
    ... AS name,
    ... AS transaction_amt
FROM ...
WHERE ...
ORDER BY ...
"""

### BEGIN SOLUTION
query_q1b = '''
SELECT 
    cmte_id AS committee_id, 
    name AS name,
    transaction_amt AS transaction_amt
FROM indiv_sample_nyc
WHERE transaction_amt > 5000000
ORDER BY transaction_amt DESC, name
'''
### END SOLUTION
pd.read_sql(query_q1b, engine)

In [ ]:
res = pd.read_sql(query_q1b, engine)
assert res.shape == (6, 3)
assert set(res.columns) == set(['committee_id', 'name', 'transaction_amt'])
assert all(res == res.sort_values('transaction_amt', ascending=False))

assert set(res['name'].unique()) == set(['HILLARY VICTORY FUND', 'C.V. STARR & CO INC', 'TRUMP, DONALD J.',
       'BLOOMBERG, MICHAEL R.', 'SOROS, GEORGE'])

### Question 1c

Of course, only looking at individual transactions could be misleading, since each contributor can make several transactions. A more interesting question is: How much did each contributor give *in total*?  Find the total transaction amounts after grouping by the name.  This time, just use `LIMIT 20` to limit your results to the top 20 total donations. Sort the results in descending order by total contribution amount. Break ties by alphabetical order of name as in the previous question.

We will be grading the query string `query_q1c`.  You may modify our template but the result should contain the same information with the same names.

The output should look something like: 


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>total_transaction_amt</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>HILLARY VICTORY FUND</td>
      <td>35379700</td>
    </tr>
    <tr>
      <th>1</th>
      <td>SOROS, GEORGE</td>
      <td>22359611</td>
    </tr>
    <tr>
      <th>2</th>
      <td>BLOOMBERG, MICHAEL R.</td>
      <td>21675924</td>
    </tr>
    <tr>
      <th>3</th>
      <td>TRUMP, DONALD J.</td>
      <td>18638557</td>
    </tr>
    <tr>
      <th>4</th>
      <td>SIMONS, JAMES H.</td>
      <td>11345900</td>
    </tr>
    <tr>
      <th>5</th>
      <td>SIMONS, JAMES</td>
      <td>11059000</td>
    </tr>
    <tr>
      <th>6</th>
      <td>C.V. STARR &amp; CO INC</td>
      <td>10000000</td>
    </tr>
    <tr>
      <th>7</th>
      <td>SINGER, PAUL</td>
      <td>9379453</td>
    </tr>
    <tr>
      <th>8</th>
      <td>SINGER, PAUL ELLIOT</td>
      <td>7400000</td>
    </tr>
    <tr>
      <th>9</th>
      <td>SHAW, DAVID E.</td>
      <td>5369500</td>
    </tr>
    <tr>
      <th>10</th>
      <td>HILLARY ACTION FUND</td>
      <td>4950000</td>
    </tr>
    <tr>
      <th>11</th>
      <td>SCHWARTZ, BERNARD L.</td>
      <td>4725600</td>
    </tr>
    <tr>
      <th>12</th>
      <td>SOROS, ALEXANDER</td>
      <td>4361524</td>
    </tr>
    <tr>
      <th>13</th>
      <td>SCHWARZMAN, STEPHEN A.</td>
      <td>3997400</td>
    </tr>
    <tr>
      <th>14</th>
      <td>MUNGER, PHILIP R.</td>
      <td>3390000</td>
    </tr>
    <tr>
      <th>15</th>
      <td>SINGER, PAUL E.</td>
      <td>3066000</td>
    </tr>
    <tr>
      <th>16</th>
      <td>SINGER, PAUL ELLIOTT</td>
      <td>3000000</td>
    </tr>
    <tr>
      <th>17</th>
      <td>SIMONS, MARILYN</td>
      <td>2531600</td>
    </tr>
    <tr>
      <th>18</th>
      <td>UNITE HERE TIP STATE &amp; LOCAL FUND</td>
      <td>2527009</td>
    </tr>
    <tr>
      <th>19</th>
      <td>C. V. STARR &amp; COMPANY, INC.</td>
      <td>2500000</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q1c = '''
SELECT 
    ... AS name,
    ... AS total_transaction_amt 
FROM indiv_sample_nyc 
GROUP BY ... 
ORDER BY ... DESC
LIMIT ...
'''
### BEGIN SOLUTION
query_q1c = '''
SELECT 
    name AS name, 
    sum(transaction_amt) AS total_transaction_amt 
FROM indiv_sample_nyc
GROUP BY name
ORDER BY total_transaction_amt DESC, name
LIMIT 20
'''
### END SOLUTION

pd.read_sql(query_q1c, engine)

In [ ]:
res = pd.read_sql(query_q1c,engine)
assert res.shape == (20, 2)
assert all(res['total_transaction_amt'] == res['total_transaction_amt'].sort_values(ascending=False))

assert set(res['name']) == set(['HILLARY VICTORY FUND', 'SOROS, GEORGE', 'BLOOMBERG, MICHAEL R.',
       'TRUMP, DONALD J.', 'SIMONS, JAMES H.', 'SIMONS, JAMES',
       'C.V. STARR & CO INC', 'SINGER, PAUL', 'SINGER, PAUL ELLIOT',
       'SHAW, DAVID E.', 'HILLARY ACTION FUND', 'SCHWARTZ, BERNARD L.',
       'SOROS, ALEXANDER', 'SCHWARZMAN, STEPHEN A.', 'MUNGER, PHILIP R.',
       'SINGER, PAUL E.', 'SINGER, PAUL ELLIOTT', 'SIMONS, MARILYN',
       'UNITE HERE TIP STATE & LOCAL FUND', 'C. V. STARR & COMPANY, INC.'])

### Question 2: Donation & Occupation

### Question 2a

We might also be interested in examining the distribution of contributors by occupation. Let's first compute how many distinct occupations occur in the dataset. You may need to use the `DISTINCT` keyword to solve the problem. Make sure to rename your result column to `count`.

In [ ]:
query_q2a = '''
SELECT 
    ...
FROM indiv_sample_nyc 
'''
### BEGIN SOLUTION
query_q2a = '''
SELECT 
    COUNT(distinct occupation) as count
FROM indiv_sample_nyc
'''
### END SOLUTION

pd.read_sql(query_q2a,engine)

In [ ]:
res = pd.read_sql(query_q2a,engine)
assert len(res) == 1
assert res.columns == ['count']

assert res["count"][0] == 14089

### Question 2b 
 Write a SQL query which computes for each occupation the number of transactions in the indiv_sample table. Display the top 20 results in descending order by count. 

We will be grading the query string `query_q2b`.  You may modify our template but the result should contain the same information with the same names.

The output should look something like: 

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>occupation</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>NOT EMPLOYED</td>
      <td>56006</td>
    </tr>
    <tr>
      <th>1</th>
      <td>RETIRED</td>
      <td>48481</td>
    </tr>
    <tr>
      <th>2</th>
      <td>ATTORNEY</td>
      <td>29537</td>
    </tr>
    <tr>
      <th>3</th>
      <td></td>
      <td>21212</td>
    </tr>
    <tr>
      <th>4</th>
      <td>MANAGING DIRECTOR</td>
      <td>12540</td>
    </tr>
    <tr>
      <th>5</th>
      <td>WRITER</td>
      <td>9445</td>
    </tr>
    <tr>
      <th>6</th>
      <td>PHYSICIAN</td>
      <td>9430</td>
    </tr>
    <tr>
      <th>7</th>
      <td>PROFESSOR</td>
      <td>9309</td>
    </tr>
    <tr>
      <th>8</th>
      <td>LAWYER</td>
      <td>9048</td>
    </tr>
    <tr>
      <th>9</th>
      <td>EXECUTIVE</td>
      <td>8376</td>
    </tr>
    <tr>
      <th>10</th>
      <td>CONSULTANT</td>
      <td>7605</td>
    </tr>
    <tr>
      <th>11</th>
      <td>TEACHER</td>
      <td>6743</td>
    </tr>
    <tr>
      <th>12</th>
      <td>INVESTMENT BANKING</td>
      <td>6313</td>
    </tr>
    <tr>
      <th>13</th>
      <td>UNEMPLOYED</td>
      <td>6115</td>
    </tr>
    <tr>
      <th>14</th>
      <td>ARTIST</td>
      <td>5826</td>
    </tr>
    <tr>
      <th>15</th>
      <td>ARCHITECT</td>
      <td>5383</td>
    </tr>
    <tr>
      <th>16</th>
      <td>HOMEMAKER</td>
      <td>4982</td>
    </tr>
    <tr>
      <th>17</th>
      <td>PARTNER</td>
      <td>4719</td>
    </tr>
    <tr>
      <th>18</th>
      <td>REAL ESTATE</td>
      <td>4025</td>
    </tr>
    <tr>
      <th>19</th>
      <td>PSYCHOTHERAPIST</td>
      <td>3876</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q2b = '''
SELECT 
    ... AS occupation, 
    ... AS count
FROM ...
...
...
LIMIT ...
'''
### BEGIN SOLUTION
query_q2b = '''
SELECT 
    occupation AS occupation, 
    count(*) AS count
FROM indiv_sample_nyc
GROUP BY occupation
ORDER BY count DESC
LIMIT 20
'''
### END SOLUTION
pd.read_sql(query_q2b, engine)

In [ ]:
res = pd.read_sql(query_q2b,engine)
assert res.shape == (20, 2)
assert set(res.columns) == set(['occupation', 'count'])
assert all(res == res.sort_values('count', ascending=False))

assert set(res['occupation']).issuperset(set(['NOT EMPLOYED', 'RETIRED', 'ATTORNEY', '', 'MANAGING DIRECTOR',
       'WRITER', 'PHYSICIAN', 'PROFESSOR', 'LAWYER', 'EXECUTIVE',
       'CONSULTANT', 'TEACHER', 'INVESTMENT BANKING', 'UNEMPLOYED',
       'ARTIST', 'ARCHITECT', 'HOMEMAKER', 'PARTNER', 'REAL ESTATE',
       'PSYCHOTHERAPIST']))


### Questions 2c
Compute the average transaction amount for each occupation, and sort them in descending order.  
Display the top 10 results.

The output should look something like: 

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>occupation</th>
      <th>amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>CHAIRMAN CEO &amp; FOUNDER</td>
      <td>1.023333e+06</td>
      <td>3</td>
    </tr>
    <tr>
      <th>1</th>
      <td>FOUNDING AND MANAGING PRINCIPAL</td>
      <td>4.500000e+05</td>
      <td>2</td>
    </tr>
    <tr>
      <th>2</th>
      <td>CHAIRMAN AND SENIOR EXECUTIVE</td>
      <td>4.000000e+05</td>
      <td>1</td>
    </tr>
    <tr>
      <th>3</th>
      <td>BIOMEDICAL RESEARCHER</td>
      <td>3.333333e+05</td>
      <td>3</td>
    </tr>
    <tr>
      <th>4</th>
      <td>OWNER, CHAIRMAN, CEO</td>
      <td>3.000000e+05</td>
      <td>2</td>
    </tr>
    <tr>
      <th>5</th>
      <td>C.E..O.</td>
      <td>2.500000e+05</td>
      <td>1</td>
    </tr>
    <tr>
      <th>6</th>
      <td>EXECUTIVE VICE-PRESAIDENT</td>
      <td>2.442000e+05</td>
      <td>1</td>
    </tr>
    <tr>
      <th>7</th>
      <td>CO-CEO &amp; CO-CIO</td>
      <td>2.000000e+05</td>
      <td>1</td>
    </tr>
    <tr>
      <th>8</th>
      <td>EXECUTIVE CO-CHAIR</td>
      <td>2.000000e+05</td>
      <td>1</td>
    </tr>
    <tr>
      <th>9</th>
      <td>CHAIRPERSON</td>
      <td>1.883000e+05</td>
      <td>8</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q2c = '''
SELECT 
    ... AS occupation, 
    ... AS avg_amount,
    ... AS count
FROM indiv_sample_nyc 
...
...
LIMIT ...
'''

### BEGIN SOLUTION
query_q2c = '''
SELECT 
    occupation AS occupation, 
    avg(transaction_amt) AS amount,
    count(*) AS count
FROM indiv_sample_nyc
GROUP BY occupation 
ORDER BY amount DESC
LIMIT 10
'''
### END SOLUTION

pd.read_sql(query_q2c, engine)

In [ ]:
res = pd.read_sql(query_q2c,engine)
assert res.shape == (10, 3)
assert all(res == res.sort_values('amount', ascending=False))

assert set(list(res["amount"].astype(int))) == set([1023333,  450000,  400000,  333333,  300000,  250000,  244200,
        200000,  200000,  188300])
assert set(list(res["occupation"])) == set(['CHAIRMAN CEO & FOUNDER', 'FOUNDING AND MANAGING PRINCIPAL',
       'CHAIRMAN AND SENIOR EXECUTIVE', 'BIOMEDICAL RESEARCHER',
       'OWNER, CHAIRMAN, CEO', 'C.E..O.', 'EXECUTIVE VICE-PRESAIDENT',
       'CO-CEO & CO-CIO', 'EXECUTIVE CO-CHAIR', 'CHAIRPERSON'])

### Questions 2d

A few of the above occupations contain 1 or 2 transactions which could lead to unreliable conclusions.  Use a `HAVING` clause to keep only occupations for which we have more than 10 transactions. 

As before, display the top 10 occupations that have the largest average total transaction amount.

Your output should look similar to: 

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>occupation</th>
      <th>amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>C.E.O.</td>
      <td>185271.886792</td>
      <td>53</td>
    </tr>
    <tr>
      <th>1</th>
      <td>BIOMEDICAL RESEARCH</td>
      <td>110563.750000</td>
      <td>40</td>
    </tr>
    <tr>
      <th>2</th>
      <td>PRESIDENT/CEO</td>
      <td>92551.541463</td>
      <td>205</td>
    </tr>
    <tr>
      <th>3</th>
      <td>FOUNDER &amp; CEO</td>
      <td>68304.626374</td>
      <td>91</td>
    </tr>
    <tr>
      <th>4</th>
      <td>PHILANTHROPIST</td>
      <td>51764.404669</td>
      <td>257</td>
    </tr>
    <tr>
      <th>5</th>
      <td>FOUNDER AND PRINCIPAL</td>
      <td>43490.909091</td>
      <td>11</td>
    </tr>
    <tr>
      <th>6</th>
      <td>CEO &amp; FOUNDER</td>
      <td>37795.266667</td>
      <td>15</td>
    </tr>
    <tr>
      <th>7</th>
      <td>GROUP PRESIDENT</td>
      <td>29775.000000</td>
      <td>19</td>
    </tr>
    <tr>
      <th>8</th>
      <td>FUND MANAGER</td>
      <td>29151.119403</td>
      <td>67</td>
    </tr>
    <tr>
      <th>9</th>
      <td>CHIEF INVESTMENT OFFICER</td>
      <td>26616.666667</td>
      <td>42</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q2d = '''
SELECT 
    ... AS occupation, 
    ... AS avg_amount,
    ... AS count
FROM indiv_sample_nyc 
GROUP BY ...
HAVING ...
ORDER BY ...
LIMIT ...
'''

### BEGIN SOLUTION
query_q2d = '''
SELECT 
    occupation AS occupation, 
    avg(transaction_amt) AS amount,
    count(*) AS count
FROM indiv_sample_nyc
GROUP BY occupation 
HAVING count(*) > 10
ORDER BY amount DESC
LIMIT 10
'''
### END SOLUTION
pd.read_sql(query_q2d, engine)

In [ ]:
res = pd.read_sql(query_q2d, engine)
assert res.shape == (10, 3)
assert all(res == res.sort_values('amount', ascending=False))

assert set(list(res["amount"].astype(int))) == set([185271, 110563,  92551,  68304,  51764,  43490,  37795,  29775,
        29151,  26616])
assert set(list(res["occupation"])) == set(['C.E.O.', 'BIOMEDICAL RESEARCH', 'PRESIDENT/CEO', 'FOUNDER & CEO',
       'PHILANTHROPIST', 'FOUNDER AND PRINCIPAL', 'CEO & FOUNDER',
       'GROUP PRESIDENT', 'FUND MANAGER', 'CHIEF INVESTMENT OFFICER'])

### Questions 2e

Let's break the total contributions down by occupation and individual (grouping on both dimensions at once).  Display the `occupation`, `name`, and total contribution amount from the `indiv_sample_nyc` table. Display the top 10 results order by the `total_amount`.

The output should look something like: 
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>occupation</th>
      <th>name</th>
      <th>total_amount</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td></td>
      <td>HILLARY VICTORY FUND</td>
      <td>35379700</td>
    </tr>
    <tr>
      <th>1</th>
      <td>EXECUTIVE</td>
      <td>BLOOMBERG, MICHAEL R.</td>
      <td>21665124</td>
    </tr>
    <tr>
      <th>2</th>
      <td>PRESIDENT/CEO</td>
      <td>TRUMP, DONALD J.</td>
      <td>18633157</td>
    </tr>
    <tr>
      <th>3</th>
      <td>CHAIRMAN</td>
      <td>SOROS, GEORGE</td>
      <td>17281093</td>
    </tr>
    <tr>
      <th>4</th>
      <td>PHILANTHROPIST</td>
      <td>SIMONS, JAMES</td>
      <td>11032000</td>
    </tr>
    <tr>
      <th>5</th>
      <td>PRESIDENT</td>
      <td>SIMONS, JAMES H.</td>
      <td>10525000</td>
    </tr>
    <tr>
      <th>6</th>
      <td></td>
      <td>C.V. STARR &amp; CO INC</td>
      <td>10000000</td>
    </tr>
    <tr>
      <th>7</th>
      <td>C.E.O.</td>
      <td>SINGER, PAUL ELLIOT</td>
      <td>5000000</td>
    </tr>
    <tr>
      <th>8</th>
      <td></td>
      <td>HILLARY ACTION FUND</td>
      <td>4950000</td>
    </tr>
    <tr>
      <th>9</th>
      <td>C.E.O.</td>
      <td>SINGER, PAUL</td>
      <td>4500000</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q2e = '''
SELECT 
    ... AS occupation,
    ... AS name,
    ... AS total_amount
FROM indiv_sample_nyc 
...
...
...
'''

### BEGIN SOLUTION
query_q2e = '''
SELECT 
    occupation AS occupation,
    name AS name,
    sum(transaction_amt) AS total_amount
FROM indiv_sample_nyc
GROUP BY occupation,name
ORDER BY total_amount DESC
LIMIT 10
'''
### END SOLUTION
pd.read_sql(query_q2e, engine)

In [ ]:
res = pd.read_sql(query_q2e, engine)
assert res.shape == (10, 3)
assert all(res == res.sort_values('total_amount', ascending=False))

assert set(list(res["total_amount"].astype(int))) == set([35379700, 21665124, 18633157, 17281093, 11032000, 10525000,
       10000000,  5000000,  4950000,  4500000])
assert set(list(res["occupation"])).issuperset(set(['', 'EXECUTIVE', 'PRESIDENT/CEO', 'CHAIRMAN', 'PHILANTHROPIST',
       'PRESIDENT', '', 'C.E.O.', '', 'C.E.O.']))

### Question 2f

Let's now examine the **average** of the **total contribution** for each person registered within an occupation. Note we will need to do this in two steps.  We will first need to compute the total contributions for each user and group and then average the totals across groups.  Use the following nested query template to answer this question. Restrict the final output to occupations that have more than 5 individuals.  Display the top 10 results order the results by the average amount in descending order and break ties with the occupation in ascending order. 

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>occupation</th>
      <th>amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>BIOMEDICAL RESEARCH</td>
      <td>737091.666667</td>
      <td>6</td>
    </tr>
    <tr>
      <th>1</th>
      <td>PRESIDENT/CEO</td>
      <td>677609.500000</td>
      <td>28</td>
    </tr>
    <tr>
      <th>2</th>
      <td>C.E.O.</td>
      <td>392776.400000</td>
      <td>25</td>
    </tr>
    <tr>
      <th>3</th>
      <td>CANDIDATE</td>
      <td>210859.285714</td>
      <td>7</td>
    </tr>
    <tr>
      <th>4</th>
      <td>CHAIRMAN AND CHIEF EXECUTIVE OFFICER</td>
      <td>159750.000000</td>
      <td>7</td>
    </tr>
    <tr>
      <th>5</th>
      <td>FOUNDER &amp; CEO</td>
      <td>144551.651163</td>
      <td>43</td>
    </tr>
    <tr>
      <th>6</th>
      <td>PHILANTHROPIST</td>
      <td>112741.118644</td>
      <td>118</td>
    </tr>
    <tr>
      <th>7</th>
      <td>FUND MANAGER</td>
      <td>102796.052632</td>
      <td>19</td>
    </tr>
    <tr>
      <th>8</th>
      <td>GROUP PRESIDENT</td>
      <td>80817.857143</td>
      <td>7</td>
    </tr>
    <tr>
      <th>9</th>
      <td>MUSIC PRODUCER</td>
      <td>63192.500000</td>
      <td>10</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q2f = '''
SELECT occupation, avg(total_amount) AS amount, count(*) AS count
FROM (
    SELECT ..., ..., ... AS total_amount
    FROM indiv_sample_nyc 
    GROUP BY ...
) AS total_contributions
GROUP BY ...
HAVING ...
ORDER BY ...
LIMIT ...
'''
### BEGIN SOLUTION
query_q2f = '''
SELECT occupation, avg(total_amount) AS amount, count(*) AS count
FROM (
    SELECT name, occupation, sum(transaction_amt) AS total_amount
    FROM indiv_sample_nyc 
    GROUP BY name, occupation
) AS total_contributions
GROUP BY occupation
HAVING count(*) > 5
ORDER BY amount DESC, occupation
LIMIT 10
'''
### END SOLUTION

pd.read_sql(query_q2f, engine)

In [ ]:
res = pd.read_sql(query_q2f,engine)
assert res.shape == (10, 3)
assert all(res == res.sort_values('amount', ascending=False))
assert set(list(res["amount"].astype(int))) == set([737091, 677609, 392776, 210859, 159750, 144551, 112741, 102796,
        80817,  63192])
assert set(list(res["occupation"])) == set(['BIOMEDICAL RESEARCH', 'PRESIDENT/CEO', 'C.E.O.', 'CANDIDATE',
       'CHAIRMAN AND CHIEF EXECUTIVE OFFICER', 'FOUNDER & CEO',
       'PHILANTHROPIST', 'FUND MANAGER', 'GROUP PRESIDENT',
       'MUSIC PRODUCER'])

### Question 3: Committees and Parties
Now let's examine the committees that appear in the `indiv_sample_nyc` table, with the information from the `comm` table. First let's take a look of the `comm` table.

In [ ]:
pd.read_sql("select * from comm limit 2", engine)

### Questions 3a
Let's first groupby the transactions in the `indiv_sample_nyc` table with `cmte_id` column (committee id), then count how many transactions and how much total amount of contribution there are for each `cmte_id`. Select the top 5 committees order by the transactions count.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>committee_id</th>
      <th>total_amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>C00401224</td>
      <td>20442777</td>
      <td>132822</td>
    </tr>
    <tr>
      <th>1</th>
      <td>C00575795</td>
      <td>25099091</td>
      <td>110657</td>
    </tr>
    <tr>
      <th>2</th>
      <td>C00000935</td>
      <td>6989835</td>
      <td>36237</td>
    </tr>
    <tr>
      <th>3</th>
      <td>C00577130</td>
      <td>2361892</td>
      <td>30504</td>
    </tr>
    <tr>
      <th>4</th>
      <td>C00586537</td>
      <td>63242107</td>
      <td>23295</td>
    </tr>
  </tbody>
</table>

In [ ]:
query_q3a='''
SELECT
    ... AS committee_id,
    ... AS total_amount,
    ... AS count
FROM indiv_sample_nyc
...
...
...
'''

### BEGIN SOLUTION
query_q3a='''
SELECT
    cmte_id AS committee_id,
    sum(transaction_amt) AS total_amount,
    count(*) AS count
FROM indiv_sample_nyc
GROUP BY cmte_id
ORDER BY count DESC
LIMIT 5
'''
### END SOLUTION

pd.read_sql(query_q3a, engine)

In [ ]:
res = pd.read_sql(query_q3a,engine)
assert res.shape == (5, 3)
assert all(res == res.sort_values('count', ascending=False))
assert set(res['total_amount'].unique()) == set([20442777, 25099091,  6989835,  2361892, 63242107])

### Questions 3b
Only showing the committee id doesn't tell us much. Let's also select the committee information by joining with the `comm` table using the `cmite_id` field.

You will extend the query we have from q3a and then perform a `JOIN` with the `comm` table. Select extra committee information such as committee name, party_affiliation, city and state from `comm`. Select the top 10 committees order by transaction count.

Your output should look like the following:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cmte_name</th>
      <th>party_affiliation</th>
      <th>city</th>
      <th>state</th>
      <th>total_amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ACTBLUE</td>
      <td></td>
      <td>SOMERVILLE</td>
      <td>MA</td>
      <td>20442777</td>
      <td>132822</td>
    </tr>
    <tr>
      <th>1</th>
      <td>HILLARY FOR AMERICA</td>
      <td>DEM</td>
      <td>NEW YORK</td>
      <td>NY</td>
      <td>25099091</td>
      <td>110657</td>
    </tr>
    <tr>
      <th>2</th>
      <td>DCCC</td>
      <td>DEM</td>
      <td>WASHINGTON</td>
      <td>DC</td>
      <td>6989835</td>
      <td>36237</td>
    </tr>
    <tr>
      <th>3</th>
      <td>BERNIE 2016</td>
      <td>DEM</td>
      <td>BURLINGTON</td>
      <td>VT</td>
      <td>2361892</td>
      <td>30504</td>
    </tr>
    <tr>
      <th>4</th>
      <td>HILLARY VICTORY FUND</td>
      <td></td>
      <td>NEW YORK</td>
      <td>NY</td>
      <td>63242107</td>
      <td>23295</td>
    </tr>
    <tr>
      <th>5</th>
      <td>DSCC</td>
      <td>DEM</td>
      <td>WASHINGTON</td>
      <td>DC</td>
      <td>7969344</td>
      <td>13690</td>
    </tr>
    <tr>
      <th>6</th>
      <td>JPMORGAN CHASE &amp; CO. FEDERAL POLITICAL ACTION ...</td>
      <td></td>
      <td>WASHINGTON</td>
      <td>DC</td>
      <td>750564</td>
      <td>13007</td>
    </tr>
    <tr>
      <th>7</th>
      <td>END CITIZENS UNITED</td>
      <td></td>
      <td>WASHINGTON</td>
      <td>DC</td>
      <td>447766</td>
      <td>12795</td>
    </tr>
    <tr>
      <th>8</th>
      <td>PFIZER INC. PAC</td>
      <td>UNK</td>
      <td>NEW YORK</td>
      <td>NY</td>
      <td>757750</td>
      <td>11166</td>
    </tr>
    <tr>
      <th>9</th>
      <td>MORGAN STANLEY POLITICAL ACTION COMMITTEE</td>
      <td></td>
      <td>NEW YORK</td>
      <td>NY</td>
      <td>783278</td>
      <td>11126</td>
    </tr>
  </tbody>
</table>

In [ ]:
query_q3b = '''
WITH cmte_count AS
(
  SELECT 
      ... AS cmte_id,
      ... AS total_amount,
      ... AS count
  FROM indiv_sample_nyc 
  ...
)
SELECT 
    ... AS cmte_name, 
    ... AS party_affiliation,
    ... AS city,
    ... AS state,
    ... AS total_amount,
    ... AS count
FROM cmte_count i JOIN comm c ON ...
...
...
'''
### BEGIN SOLUTION
query_q3b = '''
WITH cmte_count AS
(
  SELECT 
      cmte_id AS cmte_id,
      sum(transaction_amt) AS total_amount,
      count(*) AS count
  FROM indiv_sample_nyc 
  GROUP BY cmte_id 
)
SELECT 
    c.cmte_nm AS cmte_name, 
    c.cmte_pty_affiliation AS party_affiliation,
    c.cmte_city AS city,
    c.cmte_st AS state,
    i.total_amount AS total_amount,
    i.count AS count
FROM cmte_count i JOIN comm c ON c.cmte_id = i.cmte_id
ORDER BY count DESC
LIMIT 10
'''
### END SOLUTION

pd.read_sql(query_q3b, engine)

In [ ]:
res = pd.read_sql(query_q3b,engine)[["cmte_name", "count"]]
assert res.shape == (10, 2)
assert all(res == res.sort_values('count', ascending=False))
assert set(res['count'].unique()) == set([132822, 110657,  36237,  30504,  23295,  13690,  13007,  12795,
        11166,  11126])